In [1]:
#Bibliotecas
import librosa
import itertools
import numpy as np
import pandas as pd
from os import listdir
from os import chdir
from sklearn.cluster import KMeans  
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

In [2]:
#Semente
np.random.seed(seed = 42)

In [4]:
def SubSegmentacao(audio,fs,tamanho,tempo,pontos):
    D = np.abs(librosa.stft(audio))**2
    S = librosa.feature.melspectrogram(S=D)
    db = librosa.amplitude_to_db(S)
    bounds = librosa.segment.agglomerative(db, pontos)
    bound_times = librosa.frames_to_time(bounds, sr=fs)
    inicio = int(tamanho*(np.min(bound_times[4:])/tempo))
    fim    = int(tamanho*(np.max(bound_times[4:])/tempo))
    return (audio[inicio:fim])

In [5]:
def SegmentarAudio(audio,fs,n,tamanho,tempo):
    D = np.abs(librosa.stft(audio))**2
    S = librosa.feature.melspectrogram(S=D)
    db = librosa.amplitude_to_db(S)
    bounds = librosa.segment.agglomerative(db, 500)
    bound_times = librosa.frames_to_time(bounds, sr=fs)
    bound_times = np.vstack((bound_times[2:], [0]*len(bound_times[2:]))).T
    kmeans = KMeans(n_clusters=n)  
    kmeans.fit(bound_times[2:])  
    func1 = lambda x: int(tamanho*(x/tempo))
    centros =  np.array([func1(xi) for xi in kmeans.cluster_centers_[:,0]])
    centros =  np.append(centros,tamanho)
    centros.sort()
    ultimo  = 0
    retorno = []
    for i,j in zip(centros,centros[1:]):
        sl = int((i+j)/2)
        retorno.append(SubSegmentacao(audio[ultimo:sl],fs,tamanho,tempo,100))
        ultimo = sl
    return np.array(retorno)

In [6]:
scaler      = StandardScaler()
nLetras     = 4   #Número de letras que constará por arquivo
n_mfcc      = 50  #Número de componentes MFCC que irá ser extraido

#Pasta = Caminho da pasta que contém os arquivos de treinamento
def TreinarModelo(Pasta):
    chdir(Pasta)
    print('Extraindo atributo MFCC dos arquivos...')
    atributos  = []
    rotulos    = []
    arquivos = listdir()
    arquivos.sort()
    for arquivo in arquivos:
        if 'wav' in arquivo:
            audio, fs = librosa.load(arquivo,None)
            tamanho = audio.shape[0]
            tempo   = audio.shape[0]/fs
            retorno = SegmentarAudio(audio, fs, nLetras, tamanho, tempo)
            print('Extraindo MFCC do arquivo ', arquivo, '.')
            for i in range(0, nLetras): 
                retorno[i,] = librosa.util.normalize(retorno[i,],axis=0, norm=1)
                y_harm, y_per = librosa.effects.hpss(retorno[i,])
                mfcc2    =  librosa.feature.mfcc(y_harm,  sr=fs, n_mfcc=n_mfcc,hop_length=128).T
                mfcc1    =  librosa.feature.mfcc(retorno[i,],  sr=fs, n_mfcc=n_mfcc,hop_length=128).T
                valor    =  np.concatenate((mfcc1,mfcc2))
                valor    = scaler.fit_transform(valor)
                atributos.append(valor)
                rotulos = rotulos+[(arquivo[i])]*len(valor)
    print('Treinando o modelo...')
    atributos = np.vstack(atributos)
    rotulos   = np.array(rotulos)
    modelo = RandomForestClassifier(n_estimators=10,criterion="entropy",n_jobs=-1)
    modelo.fit(atributos, rotulos)
    return modelo

In [7]:
modelo = TreinarModelo('base_treinamento_I')
print('Modelo treinado!')

Extraindo atributo MFCC dos arquivos...
Extraindo MFCC do arquivo  66bx.wav .
Extraindo MFCC do arquivo  66c7.wav .
Extraindo MFCC do arquivo  66cb.wav .
Extraindo MFCC do arquivo  66cc.wav .
Extraindo MFCC do arquivo  67aa.wav .
Extraindo MFCC do arquivo  67bm.wav .
Extraindo MFCC do arquivo  6a67.wav .
Extraindo MFCC do arquivo  6ad6.wav .
Extraindo MFCC do arquivo  6c6c.wav .
Extraindo MFCC do arquivo  6dbc.wav .
Extraindo MFCC do arquivo  6hhd.wav .
Extraindo MFCC do arquivo  6m7m.wav .
Extraindo MFCC do arquivo  6n77.wav .
Extraindo MFCC do arquivo  6nhc.wav .
Extraindo MFCC do arquivo  6nxx.wav .
Extraindo MFCC do arquivo  6x67.wav .
Extraindo MFCC do arquivo  766x.wav .
Extraindo MFCC do arquivo  7676.wav .
Extraindo MFCC do arquivo  767x.wav .
Extraindo MFCC do arquivo  76ch.wav .
Extraindo MFCC do arquivo  76xc.wav .
Extraindo MFCC do arquivo  776b.wav .
Extraindo MFCC do arquivo  77cc.wav .
Extraindo MFCC do arquivo  7a6a.wav .
Extraindo MFCC do arquivo  7aaa.wav .
Extraindo 

In [8]:
def ValidacaoModelo(Pasta, Modelo):
    lstPreditos = []
    lstReais    = []
    nAcertos    = 0
    nAcertoCaracter = 0
    nQuantidade = 0
    nQuantidadeCaracter = 0
    chdir(Pasta)
    arquivos = listdir()
    arquivos.sort()
    print("Iniciando a Validação...")
    for arquivo in arquivos:
        if 'wav' in arquivo: 
            audio, fs = librosa.load(arquivo,None)
            tamanho = audio.shape[0]
            tempo   = audio.shape[0]/fs
            retorno = SegmentarAudio(audio, fs, nLetras, tamanho, tempo)
            nAcerto = 0
            nQuantidade = nQuantidade + 1
            nQuantidadeCaracter = nQuantidadeCaracter + 4
            print('Validando o arquivo ', arquivo, '.')
            for i in range(0,nLetras):
                retorno[i,] = librosa.util.normalize(retorno[i,],axis=0, norm=1)
                y_harm, y_per = librosa.effects.hpss(retorno[i,])
                mfcc2    =  librosa.feature.mfcc(y_harm,  sr=fs, n_mfcc=n_mfcc,hop_length=128).T
                mfcc1    =  librosa.feature.mfcc(retorno[i,],  sr=fs, n_mfcc=n_mfcc,hop_length=128).T
                valor    =  np.concatenate((mfcc1,mfcc2))
                valor    = scaler.fit_transform(valor)
                pred     = Modelo.predict(valor) 
                (values,counts) = np.unique(pred,return_counts=True)
                ind = np.argmax(counts)
                lstPreditos.append(values[ind])
                lstReais.append(arquivo[i])
                if values[ind] == arquivo[i]:
                    nAcerto = nAcerto + 1
                    nAcertoCaracter = nAcertoCaracter + 1
        if nAcerto == 4:
            nAcertos = nAcertos + 1
    
    print("Validação Encerrada!")
    return lstPreditos, lstReais, (nAcertos/nQuantidade), (nAcertoCaracter/nQuantidadeCaracter)

In [9]:
lstPreditos = []
lstReais    = []
vAcuraciaCAPTCHA   = 0.0
vAcuraciaCaracter   = 0.0

chdir('..')
lstPreditos, lstReais, vAcuraciaCAPTCHA, vAcuraciaCaracter  = ValidacaoModelo('base_validacao_I',modelo)

Iniciando a Validação...
Validando o arquivo  6a7h.wav .
Validando o arquivo  6ama.wav .
Validando o arquivo  6c7d.wav .
Validando o arquivo  6hcm.wav .
Validando o arquivo  6hcx.wav .
Validando o arquivo  6hnc.wav .
Validando o arquivo  6mna.wav .
Validando o arquivo  6mx6.wav .
Validando o arquivo  6x77.wav .
Validando o arquivo  6xcx.wav .
Validando o arquivo  767n.wav .
Validando o arquivo  76am.wav .
Validando o arquivo  77x7.wav .
Validando o arquivo  7a76.wav .
Validando o arquivo  7a7x.wav .
Validando o arquivo  7abb.wav .
Validando o arquivo  7adc.wav .
Validando o arquivo  7bca.wav .
Validando o arquivo  7bmx.wav .
Validando o arquivo  7bna.wav .
Validando o arquivo  7cmh.wav .
Validando o arquivo  7cnh.wav .
Validando o arquivo  7da6.wav .
Validando o arquivo  7h6d.wav .
Validando o arquivo  7hcm.wav .
Validando o arquivo  7hxb.wav .
Validando o arquivo  7m7b.wav .
Validando o arquivo  7n7x.wav .
Validando o arquivo  7nxd.wav .
Validando o arquivo  7xba.wav .
Validando o arq

In [10]:
print("Acurácia por CAPTCHA: {:.2f}".format(vAcuraciaCAPTCHA*100),"%")

print("Acurácia por Caracter: {:.2f}".format(vAcuraciaCaracter*100),"%")

Acurácia por CAPTCHA: 43.54 %
Acurácia por Caracter: 79.93 %


In [11]:
Areais    =  np.array(lstReais)
Apreditos =  np.array(lstPreditos)
print ("Matriz de Confusão\n")
print (pd.crosstab(Areais,Apreditos,rownames=['REAL'],colnames=['PREDITO'],margins=True))

Matriz de Confusão

PREDITO   6   7   a    b   c   d   h   m   n   x  All
REAL                                                 
6        55   4   0    1   3   0   0   1   0   1   65
7         0  60   0    0   0   0   0   0   0   1   61
a         0   1  61    0   0   0   0   0   0   0   62
b         0   0   0   57   0   1   0   0   0   0   58
c        11   9   0    2  42   0   0   1   0   4   69
d         2   0   0   35   0   8   0   2   1   0   48
h         0   0   1    0   0   0  69   0   0   0   70
m         0   1   0    6   0   1   0  39  15   0   62
n         0   0   0    3   0   0   0   6  38   0   47
x         3   1   0    0   1   0   0   0   0  41   46
All      71  76  62  104  46  10  69  49  54  47  588
